In [3]:
import pandas as pd
import numpy as np
import requests
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import joblib
from flask import Flask, request, jsonify
from datetime import datetime, timedelta
import os

fastforex_api_key = os.environ.get("FASTFOREX_API_KEY", "473d528421-7b5949aefb-t0lvdz") # Use environment variable
FASTFOREX_BASE_URL = "https://api.fastforex.io"

def fetch_forex_data(symbol="EURUSD"):
    base_currency = symbol[:3]
    target_currency = symbol[3:]
    end_date = datetime.now().date() - timedelta(days=1)
    start_date = end_date - timedelta(days=13)

    all_data = []
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime("%Y-%m-%d")
        url = f"{FASTFOREX_BASE_URL}/historical"
        params = {"date": date_str, "from": base_currency, "to": target_currency, "api_key": fastforex_api_key}
        response = requests.get(url, params=params)
        if response.status_code != 200:
            current_date += timedelta(days=1)
            continue
        json_data = response.json().get("results") # Corrected to access "results"
        if json_data and target_currency in json_data:
            all_data.append({"t": date_str, "close": json_data[target_currency]})
        current_date += timedelta(days=1)

    df = pd.DataFrame(all_data)
    df["t"] = pd.to_datetime(df["t"])
    df.set_index("t", inplace=True)
    df = df.resample('1h').ffill()
    return df

def compute_rsi(series: pd.Series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(50)

def add_features(df):
    if df.empty or len(df) < 14:
        print("Insufficient data for feature engineering.")
        return df
    df["returns"] = df["close"].pct_change()
    df["ma_10"] = df["close"].rolling(10).mean()
    df["volatility"] = df["returns"].rolling(10).std()
    df["rsi_14"] = compute_rsi(df["close"])
    df.dropna(inplace=True)
    df["direction"] = (df["returns"].shift(-1) > 0).astype(int)
    return df

def classify_risk(volatility):
    if volatility > 0.015:
        return "High"
    elif volatility > 0.007:
        return "Medium"
    else:
        return "Low"

def recommend_leverage(confidence, risk):
    if confidence > 0.85 and risk == "Low":
        return 100
    elif confidence > 0.7 and risk != "High":
        return 50
    elif confidence > 0.5:
        return 20
    else:
        return 5

def train_models(df):
    features = ["returns", "ma_10", "volatility", "rsi_14"]
    X = df[features]
    y = df["direction"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    pipelines = {
        "logistic_regression": LogisticRegression(max_iter=1000),
        "random_forest": RandomForestClassifier(n_estimators=100),
        "xgboost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    }
    scalers = {}
    for name, model in pipelines.items():
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        model.fit(X_train_scaled, y_train)
        joblib.dump(model, f"{name}_model.joblib")
        joblib.dump(scaler, f"{name}_scaler.joblib")
        scalers[name] = scaler

    # LSTM training
    mm = MinMaxScaler()
    X_norm = mm.fit_transform(X)
    seq_len = 10

    class ForexDataset(Dataset):
        def __init__(self, data, target):
            self.X, self.y = [], []
            for i in range(len(data) - seq_len):
                self.X.append(data[i:i+seq_len])
                self.y.append(target[i+seq_len])
            self.X = torch.tensor(self.X, dtype=torch.float32)
            self.y = torch.tensor(self.y, dtype=torch.long)
        def __len__(self):
            return len(self.X)
        def __getitem__(self, idx):
            return self.X[idx], self.y[idx]

    class LSTMModel(nn.Module):
        def __init__(self, input_size, hidden_size=64):
            super().__init__()
            self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, 2)
        def forward(self, x):
            out, _ = self.lstm(x)
            return self.fc(out[:, -1])

    dataset = ForexDataset(X_norm, y.values)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    model = LSTMModel(len(features))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(5):
        for xb, yb in loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()
    torch.save(model.state_dict(), "lstm_model.pth")
    joblib.dump(mm, "lstm_scaler.joblib")


# Flask API
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    required_features = ["returns", "ma_10", "volatility", "rsi_14"]
    if not all(k in data for k in required_features):
        return jsonify({"error": "Missing required features"}), 400

    X_input = np.array([[data[f] for f in required_features]])
    try:
        model = joblib.load("xgboost_model.joblib")
        scaler = joblib.load("xgboost_scaler.joblib")
        X_scaled = scaler.transform(X_input)
        prob = model.predict_proba(X_scaled)[0][1]
        pred = int(prob > 0.5)
    except Exception as e:
        return jsonify({"error": f"Prediction failed: {str(e)}"}), 500

    risk = classify_risk(data["volatility"])
    leverage = recommend_leverage(prob, risk)

    return jsonify({
        "direction": "Up" if pred else "Down",
        "confidence": round(prob, 4),
        "risk_level": risk,
        "recommended_leverage": leverage
    })

@app.route("/health", methods=["GET"])
def health_check():
    return jsonify({"status": "running"}), 200

if __name__ == "__main__":
    try:
        pairs = ["EURUSD", "USDJPY", "USDEUR"]  # Add any pairs you want to support here
        for pair in pairs:
            print(f"Fetching and training models for {pair}")
            df_raw = fetch_forex_data(pair)
            df_feat = add_features(df_raw)
            train_models(df_feat)
        app.run(host="0.0.0.0", port=8000)
    except Exception as e:
        print(f"Error: {e}")

Fetching and training models for EURUSD


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [09:39:16] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fetching and training models for USDJPY


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [09:39:19] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Fetching and training models for USDEUR


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [09:39:22] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
